In [1]:
import torch
import sys
import torch.nn.functional as F
from torch.cuda import amp
from spikingjelly.activation_based import functional, surrogate, neuron
from spikingjelly.activation_based.model import parametric_lif_net
from spikingjelly.datasets.dvs128_gesture import DVS128Gesture
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time
import os
import argparse
import datetime

d:\dev\anaconda3\envs\snn\lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
import numpy as np
if not hasattr(np, 'int'):
    np.int = int

In [ ]:
#空间-时间-极性一致性滤波
from typing import Dict
import numpy as np
import spikingjelly.datasets as sjds
from scipy.spatial import cKDTree

def polarity_consistent_filter_events(events: Dict, min_neighbors=1, time_window=1000):
    """
    空间-时间-极性一致性滤波
    """
    t, x, y, p = (events[key] for key in ('t', 'x', 'y', 'p'))
    features = np.stack([t // time_window, x, y], axis=1)
    tree = cKDTree(features)
    # 查询每个点的邻居索引
    neighbors = tree.query_ball_point(features, r=1)
    keep = []
    for i, idxs in enumerate(neighbors):
        # 统计极性一致的邻居数（包括自己）
        same_polarity = np.sum(p[idxs] == p[i])
        if same_polarity > min_neighbors:
            keep.append(i)
    keep = np.array(keep)
    return {
        't': t[keep],
        'x': x[keep],
        'y': y[keep],
        'p': p[keep]
    }

def integrate_events_to_2_frames_denoised2(events: Dict, H: int, W: int):
    # 用极性一致性滤波
    events = polarity_consistent_filter_events(events, min_neighbors=1, time_window=1000)
    if len(events['t']) == 0:
        return np.zeros([2, 2, H, W])
    index_split = np.random.randint(low=0, high=len(events['t']))
    frames = np.zeros([2, 2, H, W])
    t, x, y, p = (events[key] for key in ('t', 'x', 'y', 'p'))
    frames[0] = sjds.integrate_events_segment_to_frame(x, y, p, H, W, 0, index_split)
    frames[1] = sjds.integrate_events_segment_to_frame(x, y, p, H, W, index_split, len(t))
    return frames

def integrate_events_to_16_frames_by_time(events: Dict, H: int, W: int):
    if len(events['t']) == 0:
        return np.zeros([16, 2, H, W])

    events = polarity_consistent_filter_events(events, min_neighbors=1, time_window=1000)

    t = events['t']
    x = events['x']
    y = events['y']
    p = events['p']

    t_start = t[0]
    t_end = t[-1]
    total_duration = t_end - t_start
    base_delta = total_duration // 16
    remainder = total_duration % 16

    frames = np.zeros([16, 2, H, W], dtype=np.float32)

    for i in range(16):
        t_low = t_start + i * base_delta + min(i, remainder)
        t_high = t_low + base_delta + (1 if i < remainder else 0)
        mask = np.logical_and(t >= t_low, t < t_high)
        frame = sjds.integrate_events_segment_to_frame(x[mask], y[mask], p[mask], H, W)
        frames[i] = frame

    return frames

In [3]:
from spikingjelly.datasets.dvs128_gesture import DVS128Gesture

root_dir = '../datasets/DVS128Gesture'
train_set = DVS128Gesture(root_dir, train=True, data_type='event')

In [ ]:
def main():
    # python -m spikingjelly.activation_based.examples.classify_dvsg -T 16 -device cuda:0 -b 16 -epochs 64 -data-dir /datasets/DVSGesture/ -amp -cupy -opt adam -lr 0.001 -j 8

    parser = argparse.ArgumentParser(description='Classify DVS Gesture')
    parser.add_argument('-T', default=16, type=int, help='simulating time-steps')                                   #   16
    parser.add_argument('-device', default='cuda:0', help='device')                                                 #   cuda:0    
    parser.add_argument('-b', default=16, type=int, help='batch size')                                              #   16
    parser.add_argument('-epochs', default=32, type=int, metavar='N',
                        help='number of total epochs to run')                                                       #   32
    parser.add_argument('-j', default=8, type=int, metavar='N',
                        help='number of data loading workers (default: 4)')                                         #   8
    parser.add_argument('-data-dir', type=str, help='root dir of DVS Gesture dataset',default='../datasets/DVS128Gesture')
    parser.add_argument('-out-dir', type=str, default='./logs', help='root dir for saving logs and checkpoint')
    parser.add_argument('-resume', type=str, help='resume from the checkpoint path')
    parser.add_argument('-amp', action='store_true', help='automatic mixed precision training',default= True)       #   amp
    parser.add_argument('-cupy', action='store_true', help='use cupy backend', default=True)                        #   cupy
    parser.add_argument('-opt', type=str, help='use which optimizer. SDG or Adam', default='adam')                  #   Adam
    parser.add_argument('-momentum', default=0.9, type=float, help='momentum for SGD')                              #   momentum = 0.9
    parser.add_argument('-lr', default=0.001, type=float, help='learning rate')                                     #   lr = 0.001
    parser.add_argument('-channels', default=128, type=int, help='channels of CSNN')                                #   128

    args, _ = parser.parse_known_args()  # 忽略 Jupyter 自动添加的 -f 参数
    print(args)

    net = parametric_lif_net.DVSGestureNet_RAW(channels=args.channels, spiking_neuron=neuron.LIFNode, surrogate_function=surrogate.ATan(), detach_reset=True)

    functional.set_step_mode(net, 'm')
    if args.cupy:
        functional.set_backend(net, 'cupy', instance=neuron.LIFNode)

    print(net)


    net.to(args.device)

    # 对模型进行减枝
    import torch.nn.utils.prune as prune
    for name, module in net.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=0.2)

    train_set = DVS128Gesture(root=args.data_dir, 
                          train=True, 
                          data_type='frame', 
                          split_by='number', 
                          custom_integrate_function=integrate_events_to_16_frames_by_time)
    # train_set = DVS128Gesture(root=args.data_dir, train=True, data_type='frame', frames_number=args.T, split_by='number')
    test_set = DVS128Gesture(root=args.data_dir, 
                          train=False, 
                          data_type='frame', 
                          split_by='number', 
                          custom_integrate_function=integrate_events_to_16_frames_by_time)
    # test_set = DVS128Gesture(root=args.data_dir, train=False, data_type='frame', frames_number=args.T, split_by='number')



    train_data_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        batch_size=args.b,
        shuffle=True,
        drop_last=True,
        num_workers=args.j,
        pin_memory=True,
        prefetch_factor=2,  # 2 让每个worker获取更多batch
    )

    test_data_loader = torch.utils.data.DataLoader(
        dataset=test_set,
        batch_size=args.b,
        shuffle=True,
        drop_last=False,
        num_workers=args.j,
        pin_memory=True,
        prefetch_factor=2,  # 2 让每个worker获取更多batch
    )


    scaler = None
    if args.amp:
        scaler = amp.GradScaler()

    start_epoch = 0
    max_test_acc = -1

    optimizer = None
    if args.opt == 'sgd':
        optimizer = torch.optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum)
    elif args.opt == 'adam':
        optimizer = torch.optim.Adam(net.parameters(), lr=args.lr)
    else:
        raise NotImplementedError(args.opt)

    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)

    if args.resume:
        checkpoint = torch.load(args.resume, map_location='cpu')
        net.load_state_dict(checkpoint['net'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        start_epoch = checkpoint['epoch'] + 1
        max_test_acc = checkpoint['max_test_acc']

    out_dir = os.path.join(args.out_dir, f'T{args.T}_b{args.b}_{args.opt}_lr{args.lr}_c{args.channels}')

    if args.amp:
        out_dir += '_amp'

    if args.cupy:
        out_dir += '_cupy'
    
    out_dir += '_datapipline'

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        print(f'Mkdir {out_dir}.')

    writer = SummaryWriter(out_dir, purge_step=start_epoch)
    with open(os.path.join(out_dir, 'args.txt'), 'w', encoding='utf-8') as args_txt:
        args_txt.write(str(args))
        args_txt.write('\n')
        args_txt.write(' '.join(sys.argv))

    for epoch in range(start_epoch, args.epochs):
        start_time = time.time()
        net.train()
        train_loss = 0
        train_acc = 0
        train_samples = 0
        for frame, label in train_data_loader:
            optimizer.zero_grad()
            frame = frame.to(args.device)
            frame = frame.transpose(0, 1)  # [N, T, C, H, W] -> [T, N, C, H, W]
            label = label.to(args.device)
            label_onehot = F.one_hot(label, 11).float()

            if scaler is not None:
                with amp.autocast():
                    out_fr = net(frame).mean(0)
                    loss = F.mse_loss(out_fr, label_onehot)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                out_fr = net(frame).mean(0)
                loss = F.mse_loss(out_fr, label_onehot)
                loss.backward()
                optimizer.step()

            train_samples += label.numel()
            train_loss += loss.item() * label.numel()
            train_acc += (out_fr.argmax(1) == label).float().sum().item()

            functional.reset_net(net)

        train_time = time.time()
        train_speed = train_samples / (train_time - start_time)
        train_loss /= train_samples
        train_acc /= train_samples

        writer.add_scalar('train_speed', train_speed, epoch)
        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('train_acc', train_acc, epoch)
        lr_scheduler.step()

        net.eval()
        test_loss = 0
        test_acc = 0
        test_samples = 0
        with torch.no_grad():
            for frame, label in test_data_loader:
                frame = frame.to(args.device)
                frame = frame.transpose(0, 1)  # [N, T, C, H, W] -> [T, N, C, H, W]
                label = label.to(args.device)
                label_onehot = F.one_hot(label, 11).float()
                out_fr = net(frame).mean(0)
                loss = F.mse_loss(out_fr, label_onehot)
                test_samples += label.numel()
                test_loss += loss.item() * label.numel()
                test_acc += (out_fr.argmax(1) == label).float().sum().item()
                functional.reset_net(net)
        test_time = time.time()
        test_speed = test_samples / (test_time - train_time)
        test_loss /= test_samples
        test_acc /= test_samples
        writer.add_scalar('test_speed', test_speed, epoch)
        writer.add_scalar('test_loss', test_loss, epoch)
        writer.add_scalar('test_acc', test_acc, epoch)

        save_max = False
        if test_acc > max_test_acc:
            max_test_acc = test_acc
            save_max = True

        checkpoint = {
            'net': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'epoch': epoch,
            'max_test_acc': max_test_acc
        }

        if save_max:
            torch.save(checkpoint, os.path.join(out_dir, 'checkpoint_max.pth'))

        torch.save(checkpoint, os.path.join(out_dir, 'checkpoint_latest.pth'))

        print(args)
        print(out_dir)
        print(f'epoch = {epoch}, train_loss ={train_loss: .4f}, train_acc ={train_acc: .4f}, test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}, max_test_acc ={max_test_acc: .4f}')
        print(f'train speed ={train_speed: .4f} images/s, test speed ={test_speed: .4f} images/s')
        print(f'escape time = {(datetime.datetime.now() + datetime.timedelta(seconds=(time.time() - start_time) * (args.epochs - epoch))).strftime("%Y-%m-%d %H:%M:%S")}\n')


if __name__ == '__main__':
    main()

Namespace(T=16, device='cuda:0', b=16, epochs=32, j=8, data_dir='../datasets/DVS128Gesture', out_dir='./logs', resume=None, amp=True, cupy=True, opt='adam', momentum=0.9, lr=0.001, channels=128)
DVSGestureNet(
  (conv_fc): Sequential(
    (0): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, step_mode=m)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, step_mode=m)
    (2): LIFNode(
      v_threshold=1.0, v_reset=0.0, detach_reset=True, step_mode=m, backend=cupy, tau=2.0
      (surrogate_function): ATan(alpha=2.0, spiking=True)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False, step_mode=m)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, step_mode=m)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, step_mode=m)
    (6): LIFNode(
      v_threshold=1.0, v_reset=0.0, detach_reset=True, step_mode=m, ba

C:\Users\jiaryWin\AppData\Local\Temp\ipykernel_15688\3590443607.py:70: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
C:\Users\jiaryWin\AppData\Local\Temp\ipykernel_15688\3590443607.py:127: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Namespace(T=16, device='cuda:0', b=16, epochs=32, j=8, data_dir='../datasets/DVS128Gesture', out_dir='./logs', resume=None, amp=True, cupy=True, opt='adam', momentum=0.9, lr=0.001, channels=128)
./logs\T16_b16_adam_lr0.001_c128_amp_cupy_datapipline
epoch = 0, train_loss = 0.0678, train_acc = 0.3673, test_loss = 0.0606, test_acc = 0.4583, max_test_acc = 0.4583
train speed = 7.1299 images/s, test speed = 9.2047 images/s
escape time = 2025-06-10 13:58:32

Namespace(T=16, device='cuda:0', b=16, epochs=32, j=8, data_dir='../datasets/DVS128Gesture', out_dir='./logs', resume=None, amp=True, cupy=True, opt='adam', momentum=0.9, lr=0.001, channels=128)
./logs\T16_b16_adam_lr0.001_c128_amp_cupy_datapipline
epoch = 1, train_loss = 0.0464, train_acc = 0.6130, test_loss = 0.0512, test_acc = 0.5694, max_test_acc = 0.5694
train speed = 6.9945 images/s, test speed = 11.0047 images/s
escape time = 2025-06-10 13:57:29



KeyboardInterrupt: 